# Análise Exploratória

Este notebook apresenta a etapa inicial de análise exploratória do Steam Games Dataset, disponibilizado em:
https://www.kaggle.com/datasets/fronkongames/steam-games-dataset
.

Para assegurar a reprodutibilidade e organização do projeto, todos os arquivos utilizados na análise foram previamente armazenados no diretório `/datasets`.

---

## Configuração do Ambiente

As bibliotecas necessárias são carregadas e algumas configurações são ajustadas para facilitar o tratamento e a visualização dos dados. Em particular, amplia-se o limite de caracteres dos campos CSV, evitando truncamento de linhas extensas, e permite-se que o pandas exiba todas as colunas do DataFrame durante a inspeção.

In [8]:
import csv
import sys
import pandas

# Remove limite padrão de tamanho de campos em arquivos CSV
csv.field_size_limit(sys.maxsize)

# Permite a visualização de todas as colunas do DataFrame
pandas.set_option('display.max_columns', None)

dataset_csv_path = "./datasets/games.csv"
dataset_csv_fixed_path = "./datasets/games_fixed.csv"

## Correção do Dataset

O arquivo original (`games.csv`) apresenta inconsistências estruturais que impedem sua leitura direta pelo pandas. Em particular, o número de colunas definido no cabeçalho não corresponde ao número de campos presentes em algumas linhas do arquivo. Esse tipo de problema é comum em bases de dados grandes e heterogêneas oriundas de web scraping ou integrações automatizadas.

Durante a inspeção inicial, identificou-se que a coluna `DiscountDLC count` estava incorretamente delimitada, ocasionando uma fusão indevida entre dois atributos distintos. Assim, procedeu-se à correção manual do cabeçalho, assegurando a correta separação entre as colunas `discount` e `dlc_count`.

Além disso, os nomes das colunas foram normalizados para um formato mais adequado ao processamento em Python: todos os caracteres foram convertidos para minúsculas, acentos e espaços foram removidos ou substituídos, e adotou-se o padrão `snake_case`.

In [9]:
# Exibe as primeiras linhas do arquivo original para inspeção
with open(dataset_csv_path, 'r', encoding='utf-8') as f:
    for i in range(5):
        print(f.readline())

# Leitura completa do arquivo
with open(dataset_csv_path, 'r', encoding='utf-8') as f:
    linhas = f.readlines()

# Correção do cabeçalho
header = linhas[0]

# Ajuste específico da coluna problemática
header = header.replace('DiscountDLC count', 'Discount,DLC count')

# Normalização dos nomes das colunas
colunas = header.strip().split(',')
colunas_normalizadas = [
    c.strip().lower().replace(" ", "_")
    for c in colunas
]

# Reconstrução do cabeçalho corrigido
linhas[0] = ",".join(colunas_normalizadas) + "\n"

# Salvamento do arquivo corrigido
with open(dataset_csv_fixed_path, 'w', encoding='utf-8') as f:
    f.writelines(linhas)

print("✅ Arquivo corrigido salvo como:", dataset_csv_fixed_path)
print("📝 Colunas normalizadas:", colunas_normalizadas)

AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DiscountDLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies

20200,"Galactic Bowling","Oct 21, 2008","0 - 20000",0,0,19.99,0,0,"Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player competition while being submerged in a unique new universe filled with over-the-top humor, wild characters, unique levels, and addictive game play. The title is aimed at players of all ages and skill sets. Through accessible and intuitive controls and game-play, Galactic Bowling allows you to j

## Carregamento do Dataset

Após a correção estrutural, o conjunto de dados é carregado no pandas para dar início à análise exploratória. Utiliza-se o parâmetro `engine="python"` para ampliar a robustez na leitura de arquivos CSV possivelmente irregulares, e define-se a codificação `"utf-8-sig"` para garantir a correta interpretação dos caracteres especiais.

In [10]:
games_dataset = pandas.read_csv(
  dataset_csv_fixed_path,
  sep=",",
  quotechar='"',
  quoting=csv.QUOTE_MINIMAL,
  engine="python",
  encoding="utf-8-sig",
)

## Análise inicial